In [16]:
#!pip install utils
#!pip install torch
#!pip install minisom
#!pip install pyclustering
#!pip install wget 
#!pip uninstall scikit-learn
#!pip install scikit-learn==1.2.2
#!pip install gower
#!pip install kmodes
#!pip install kneebow
#!pip install ucimlrepo
#!pip install --upgrade pandas

In [17]:
## Libraries
import logging
import numpy as np
import pandas as pd
import os
import time
from joblib import Parallel, delayed
from utils import *
from utils_cascade import *

# Configurando o nível de registro (pode ser DEBUG, INFO, WARNING, ERROR, ou CRITICAL)
logging.basicConfig(level=logging.INFO)

Pegando uma base para o estudo

In [18]:
iris_df = pd.read_parquet('C:/Users/DELL/imput-cascate/docs/source/Dados/iris_df(teste).parquet')
#wine_df = pd.read_parquet('C:/Users/DELL/imput-cascate/docs/source/Dados/wine_df.parquet')
#pima_df = pd.read_parquet('C:/Users/DELL/imput-cascate/docs/source/Dados/pima_df.parquet')
#boston_df = pd.read_parquet('C:/Users/DELL/imput-cascate/docs/source/Dados/boston_df.parquet').drop(columns=['RAD'])
#abalone_df = pd.read_parquet('C:/Users/DELL/imput-cascate/docs/source/Dados/abalone_df.parquet').drop(columns=['Sex'])
#california_df = pd.read_parquet('C:/Users/DELL/imput-cascate/docs/source/Dados/california_df.parquet')
#gamma_df = pd.read_parquet('C:/Users/DELL/imput-cascate/docs/source/Dados/gamma_df.parquet')
#electric_df = pd.read_parquet('C:/Users/DELL/imput-cascate/docs/source/Dados/electric_df.parquet')
#yellow_df = pd.read_parquet('C:/Users/DELL/imput-cascate/docs/source/Dados/yellow_df.parquet')

In [19]:
df = iris_df
#df = wine_df
#df = pima_df
#df = boston_df
#df = abalone_df
#df = california_df
#df = gamma_df
#df = electric_df
#df = yellow_df

In [20]:
print('Linhas  :', df.iloc[:,0].count())
print('Colunas :', len(df.columns))

Linhas  : 150
Colunas : 4


Definindo os parâmetros

In [21]:
# Ler os parâmetros do arquivo CSV
params = pd.read_csv("C:/Users/DELL/imput-cascate/docs/source/params_teste.csv", sep = ';', decimal=',').fillna(0)

# params_df = params_df.sort_values(by=['dados','mec_ausencia','pct_ausencia','cluster_algorithm','n_clusters_knn','order_cluster','order_column','dim_x_som',
#                                       'dim_y_som','sigma_som','lr_som','max_iter_som','n_clusters_kmd','n_clusters_aggcl','affinity_aggcl','metric_dbscan',
#                                       'min_samples_dbscan','eps_dbscan'], 
#                                   ascending=[True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True])

C:\Users\DELL\AppData\Local\Temp\ipykernel_26656\4232804322.py:2: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  params = pd.read_csv("C:/Users/DELL/imput-cascate/docs/source/params_teste.csv", sep = ';', decimal=',').fillna(0)


In [22]:
## Filtrando por dados
params_df = params.loc[(params['dados'] == 'Iris')]  

In [23]:
params_df

,idx_simulacao,dados,mec_ausencia,pct_ausencia,cluster_algorithm,dim_x_som,dim_y_som,sigma_som,lr_som,max_iter_som,n_clusters_kmd,n_clusters_aggcl,affinity_aggcl,metric_dbscan,min_samples_dbscan,eps_dbscan,order_cluster,order_column,n_clusters_knn
0,110001,Iris,MAR,0.1,som,1.0,8.0,0.5,0.7,50.0,0.0,0.0,0,0,0.0,0.0,fieldLessMissing,lessCorrelation,1
1,110002,Iris,MAR,0.1,som,1.0,8.0,0.5,0.7,50.0,0.0,0.0,0,0,0.0,0.0,fieldLessMissing,lessMissing,1
2,110003,Iris,MAR,0.1,som,1.0,8.0,0.5,0.7,50.0,0.0,0.0,0,0,0.0,0.0,fieldLessMissing,moreCorrelation,1
3,110004,Iris,MAR,0.1,som,1.0,8.0,0.5,0.7,50.0,0.0,0.0,0,0,0.0,0.0,fieldLessMissing,moreMissing,1
4,110005,Iris,MAR,0.1,som,1.0,8.0,0.5,0.7,50.0,0.0,0.0,0,0,0.0,0.0,fieldLessMissing,noSort,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54715,1340956,Iris,MCAR,0.3,dbscan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,precomputed,10.0,0.5,tupleMoreMissing,noSort,10
54716,1340957,Iris,MCAR,0.3,dbscan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,precomputed,15.0,0.5,tupleMoreMissing,noSort,10
54717,1340958,Iris,MCAR,0.3,dbscan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,precomputed,20.0,0.5,tupleMoreMissing,noSort,10
54718,1340959,Iris,MCAR,0.3,dbscan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,precomputed,35.0,0.5,tupleMoreMissing,noSort,10


## Imputação Cascata Paralelizada

In [24]:
import pandas as pd
from joblib import Parallel, delayed
import time

def preprocess_data(params_df, df):
    cache = {}
    for _, row in params_df.iterrows():
        params = row.to_dict()
        mec_ausencia = params['mec_ausencia']
        pct_ausencia = params['pct_ausencia']
        chave = (mec_ausencia, pct_ausencia)
        
        if chave not in cache:
            dados, _, _, df_incompleto = eraser_with_params(df.copy(), params)

            df_g1_mean, df_g1_median, time_g1 = univariate_imputation_with_params(df_incompleto)
            avg_rmse_media, avg_sim_error_media, correl_bias_media = error_metrics(df.copy(), df_g1_mean)
            avg_rmse_mediana, avg_sim_error_mediana, correl_bias_mediana = error_metrics(df.copy(), df_g1_median)

            cache[chave] = (df_incompleto, avg_rmse_media, avg_sim_error_media, correl_bias_media,
                            avg_rmse_mediana, avg_sim_error_mediana, correl_bias_mediana, time_g1)

    return cache

def process_cascade_imputation(row, df, cache):
    params = row.to_dict()
    mec_ausencia = params['mec_ausencia']
    pct_ausencia = params['pct_ausencia']
    chave = (mec_ausencia, pct_ausencia)

    df_incompleto, avg_rmse_media, avg_sim_error_media, correl_bias_media, \
    avg_rmse_mediana, avg_sim_error_mediana, correl_bias_mediana, time_g1 = cache[chave]

    idx_simulacao, df_g2_imput, time_g2 = cascade_imputation_with_params(df_incompleto.copy(), params)
    avg_rmse_cascata, avg_sim_error_cascata, correl_bias_cascata = error_metrics(df.copy(), df_g2_imput)

    return pd.DataFrame({
        "idx_simulacao": [idx_simulacao],
        "avg_rmse_media": [avg_rmse_media],
        "avg_sim_error_media": [avg_sim_error_media],
        "correl_bias_media": [correl_bias_media],
        "avg_rmse_mediana": [avg_rmse_mediana],
        "avg_sim_error_mediana": [avg_sim_error_mediana],
        "correl_bias_mediana": [correl_bias_mediana],
        "time_g1": [time_g1],
        "avg_rmse_cascata": [avg_rmse_cascata],
        "avg_sim_error_cascata": [avg_sim_error_cascata],
        "correl_bias_cascata": [correl_bias_cascata],
        "time_g2": [time_g2]
    })


cache = preprocess_data(params_df, df)

start_parallel = time.time()
results = Parallel(n_jobs=os.cpu_count())(delayed(process_cascade_imputation)(row, df, cache) for _, row in params_df.iterrows())
final_results_df = pd.concat(results, ignore_index=True)
end_parallel = time.time()

print("O código rodou em: ", end_parallel - start_parallel) 

INFO:root:Iniciando simulação 110001...
INFO:root:Eraser do MAR_0.1 concluído. Tempo decorrido: 0m.0s
INFO:root:Inicio da imputação univariada.
c:\Users\DELL\imput-cascate\notebooks\utils_cascade.py:1156: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_g1_median[name].fillna(median, inplace = True)
INFO:root:Imputação univariada concluída! Tempo decorrido: 0m.0s
INFO:root:Iniciando simulação 210001...
INFO:root:Eraser do MAR_0.2 concluído. Tempo decorrido: 0m.0s
INFO:root:Inicio da imputação univariada.
c:\Users\DELL\imput-ca

O código rodou em:  21749.49968290329


In [25]:
# Salvar em um arquivo CSV
nome_resultado = f'Resultado_simulacao_iris'
final_results_df.to_csv(f"C:/Users/DELL/imput-cascate/docs/source/Resultado simulações/{nome_resultado}.csv", index=False) 

In [26]:
#final_results_df

In [27]:
#aux = pd.read_csv("C:/Users/DELL/imput-cascate/docs/source/Resultado simulações/Resultado_simulacao_wine.csv")
#aux

In [28]:
### Tempo rodando
# Iris : 362 min
# Wine : 857 min -> 14h
# Pima : 2152m min -> 36h